# Task 2: Price a Commodity Storage Contract

Prototype pricing model for gas storage contracts.

In [ ]:

import pandas as pd
from datetime import datetime

def price_storage_contract(
    price_df,
    injections,
    withdrawals,
    injection_rate,
    withdrawal_rate,
    max_storage,
    storage_cost_per_month,
    inj_wd_cost_per_unit
):
    price_df['Date'] = pd.to_datetime(price_df['Date'])
    price_df.set_index('Date', inplace=True)

    storage_level = 0
    cash_flow = 0
    all_dates = []

    for date, volume in injections:
        date = pd.to_datetime(date)
        if volume > injection_rate:
            raise ValueError("Injection rate exceeded")
        if storage_level + volume > max_storage:
            raise ValueError("Storage capacity exceeded")
        price = price_df.loc[date, 'Price']
        cash_flow -= price * volume
        cash_flow -= inj_wd_cost_per_unit * volume
        storage_level += volume
        all_dates.append(date)

    for date, volume in withdrawals:
        date = pd.to_datetime(date)
        if volume > withdrawal_rate:
            raise ValueError("Withdrawal rate exceeded")
        if volume > storage_level:
            raise ValueError("Not enough gas in storage")
        price = price_df.loc[date, 'Price']
        cash_flow += price * volume
        cash_flow -= inj_wd_cost_per_unit * volume
        storage_level -= volume
        all_dates.append(date)

    total_months = (
        (max(all_dates).year - min(all_dates).year) * 12 +
        (max(all_dates).month - min(all_dates).month) + 1
    )
    cash_flow -= storage_cost_per_month * total_months

    return cash_flow


In [ ]:

# Example Test Case
price_df = pd.read_csv("/mnt/data/Nat_Gas.csv")

injections = [("2023-06-01", 1_000_000)]
withdrawals = [("2023-12-01", 1_000_000)]

value = price_storage_contract(
    price_df,
    injections,
    withdrawals,
    injection_rate=1_500_000,
    withdrawal_rate=1_500_000,
    max_storage=2_000_000,
    storage_cost_per_month=100_000,
    inj_wd_cost_per_unit=0.01
)

print(f"Contract Value: ${value:,.0f}")
